## Задание № 1

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

**Дана таблица default.peas:**

|Название атрибута|Тип атрибута|Смысловое значение                       |
|-----------------|------------|-----------------------------------------|
|st_id	          |int	       |ID ученика                               |
|timest	          |timestamp   |Время решения карточки                   |
|correct	      |bool	       |Правильно ли решена горошина?            |
|subject	      |text	       |Дисциплина, в которой находится горошина |


Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

In [8]:
# Чтобы проверить запросы на работоспособность, предлагается поднять БД на docker
# Команда для запуска БД: docke-compose up
# Чтобы она сработала, нужно быть в дирекции с файлом docke-compose.yaml

import psycopg2
import pandas as pd

conn = psycopg2.connect(dbname='default',
                        host="localhost",
                        user="admin",
                        password="admin",
                        port='5432')

conn.autocommit = True
cursor = conn.cursor()

### Ответ:

In [18]:
query = """
SELECT
    COUNT(sq.st_id) AS number_of_hard_studs
FROM
    (SELECT st_id
     FROM peas
     GROUP BY st_id
     HAVING SUM(CAST (correct AS int)) >= 20) AS sq
"""
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,number_of_hard_studs
0,136


## Задание № 2

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

**Дана таблицы: default.peas (см. выше), default.studs:**

|Название атрибута|Тип атрибута|Смысловое значение                 |
|-----------------|------------|-----------------------------------|
|st_id	          |int	       |ID ученика                         |
|test_grp	      |text	       |Метка ученика в данном эксперименте|

**и default.final_project_check:**

|Название атрибута|	Тип атрибута|	Смысловое значение|
|-----------------|-------------|---------------------|
|st_id            |int 	        |ID ученика
|sale_time        |timestamp	|Время покупки
|money            |int          |Цена, по которой приобрели данный курс
|subject          |text         | 	

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:
- ARPU 
- ARPAU 
- CR в покупку 
- СR активного пользователя в покупку 
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

ARPU считается относительно всех пользователей, попавших в группы.

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.

Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

## Ответ:

In [42]:
query = """
SELECT
    ARPU.test_grp AS test_grp,
    ARPU,
    ARPAU,
    CR,
    CRa,
    CRm
FROM
    -- ARPU ----------------------------------------------------------------------------------------------
    (SELECT
        test_grp,
        ROUND(SUM(money) / COUNT(DISTINCT st_id)) AS ARPU
    FROM final_project_check
    LEFT JOIN studs USING (st_id)
    GROUP BY test_grp) AS ARPU
    
    -- ARPAU ---------------------------------------------------------------------------------------------
    LEFT JOIN (
            SELECT
                test_grp,
                ROUND(SUM(money) / COUNT(DISTINCT st_id)) AS ARPAU
            FROM final_project_check
            LEFT JOIN studs USING (st_id)
            WHERE st_id in (SELECT st_id
                            FROM peas
                            GROUP BY st_id
                            HAVING SUM(CAST (correct AS int)) > 10)
            GROUP BY test_grp) AS ARPAU
        ON ARPU.test_grp = ARPAU.test_grp
    
    -- CR ------------------------------------------------------------------------------------------------
    LEFT JOIN (SELECT
                   test_grp,
                   ROUND(check_users / task_users * 100, 2) AS CR
               FROM
                   (SELECT
                        test_grp,
                        CAST (COUNT(DISTINCT st_id) AS numeric) AS task_users
                    FROM peas
                    LEFT JOIN studs USING (st_id)
                    GROUP BY test_grp) AS tasks
               LEFT JOIN (SELECT
                              test_grp,
                              CAST (COUNT(DISTINCT st_id) AS numeric) AS check_users
                          FROM final_project_check
                          LEFT JOIN studs USING (st_id)
                          GROUP BY test_grp) AS checks
                   USING (test_grp)) AS CR
        ON ARPU.test_grp = CR.test_grp
    
    -- CRa ----------------------------------------------------------------------------------------------
    LEFT JOIN (
               SELECT
                   test_grp,
                   ROUND(check_users / task_users * 100, 2) AS CRa
               FROM
                   (SELECT
                        test_grp,
                        CAST (COUNT(DISTINCT st_id) AS numeric) AS task_users
                    FROM peas
                    LEFT JOIN studs USING (st_id)
                    WHERE st_id in (SELECT st_id
                            FROM peas
                            GROUP BY st_id
                            HAVING SUM(CAST (correct AS int)) > 10)
                    GROUP BY test_grp) AS tasks
               LEFT JOIN (SELECT
                              test_grp,
                              CAST (COUNT(DISTINCT st_id) AS numeric) AS check_users
                          FROM final_project_check
                          LEFT JOIN studs USING (st_id)
                          WHERE st_id in (SELECT st_id
                                          FROM peas
                                          GROUP BY st_id
                                          HAVING SUM(CAST (correct AS int)) > 10)
                          GROUP BY test_grp) AS checks
                   USING (test_grp)) AS CRa
        ON ARPU.test_grp = CRa.test_grp
    
    -- CRm ----------------------------------------------------------------------------------------------
    LEFT JOIN (
               SELECT test_grp, ROUND(check_users / task_users * 100, 2) AS CRm
               FROM
                   (SELECT
                       test_grp,
                       CAST (COUNT(DISTINCT st_id) AS numeric) AS task_users
                    FROM peas
                    LEFT JOIN studs USING (st_id)
                    WHERE st_id in (SELECT st_id
                            FROM peas
                            WHERE subject = 'Math'
                            GROUP BY st_id
                            HAVING SUM(CAST (correct AS int)) > 2)
                    GROUP BY test_grp) AS tasks
               LEFT JOIN (SELECT
                              test_grp,
                              CAST (COUNT(DISTINCT st_id) AS numeric) AS check_users
                          FROM final_project_check
                          LEFT JOIN studs USING (st_id)
                          WHERE st_id in (SELECT st_id
                                          FROM peas
                                          WHERE subject = 'Math'
                                          GROUP BY st_id
                                          HAVING SUM(CAST (correct AS int)) > 2)
                          GROUP BY test_grp) AS checks
                   USING (test_grp)) AS CRm
        ON ARPU.test_grp = CRm.test_grp;
"""
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,test_grp,arpu,arpau,cr,cra,crm
0,control,92333.0,94285.0,9.09,11.02,10.20
1,pilot,106093.0,114200.0,22.38,26.04,14.29
